# **1. Configuración del Ambiente**


---

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
global df_traffic, resultados, modelo, modelo_clasificacion

# **2. Creación de Modelo de Gradient Boosted Regressor**


---

In [3]:
df_traffic = pd.read_csv('Classifier.csv', sep=';')
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   visitNumber         12283 non-null  int64  
 1   browser             12283 non-null  int64  
 2   operatingSystem     12283 non-null  int64  
 3   deviceCategory      12283 non-null  int64  
 4   continent           12283 non-null  int64  
 5   country             12283 non-null  int64  
 6   city                12283 non-null  int64  
 7   networkDomain       12283 non-null  int64  
 8   source              12283 non-null  int64  
 9   medium              12283 non-null  int64  
 10  keyword             12283 non-null  int64  
 11  referralPath        12283 non-null  int64  
 12  adContent           12283 non-null  int64  
 13  pageviews           12283 non-null  int64  
 14  newVisits           12283 non-null  float64
 15  transactionRevenue  12283 non-null  float64
 16  year

In [4]:
# Dividir los datos en conjunto de entrenamiento y prueba
X = df_traffic.drop(columns=['transactionRevenue'])
y = df_traffic['transactionRevenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creación del modelo Gradient Boosted Trees
gbt_model = GradientBoostingRegressor()
gbt_model.fit(X_train, y_train)

y_pred = gbt_model.predict(X_test)

# Evaluación del modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 289.57522509027825


In [5]:
#definiendo el K - número de subconjuntos
cv = KFold(n_splits = 5, shuffle=True)

In [6]:
accuracy_cross_val = cross_val_score(estimator = gbt_model,
                                       X = X_train,
                                       y = y_train,
                                       scoring = 'neg_mean_squared_error',
                                       cv = cv)
print('Accuracy promedio de mse: {}'.format(round(accuracy_cross_val.mean(), 3)))

Accuracy promedio de mse: -389.25


In [7]:
# Selección de hiperparámetros para la regresión lineal
param_gbt = {
    'loss': ['squared_error', 'huber'],  # 'squared_error' es adecuado para la regresión
    'learning_rate': [0.01, 0.05, 0.1],  # Un learning rate bajo puede ayudar a evitar sobreajuste
    'n_estimators': [100, 200, 300],  # Número moderado de estimadores para evitar sobreajuste
    'subsample': [0.8, 1.0],  # Utilizar todos los datos o una fracción alta puede ser beneficioso
    'max_depth': [3, 5, 7],  # Profundidad moderada para evitar sobreajuste
    'min_samples_split': [2, 5, 10],  # Número bajo para asegurar que haya suficientes muestras para dividir
    'min_samples_leaf': [1, 2, 4],  # Número bajo para permitir más flexibilidad en las hojas
    'max_features': ['sqrt', None],  # 'sqrt' puede ser útil para reducir la varianza
    'ccp_alpha': [0.0, 0.1],  # Parámetro de poda, puede ayudar a controlar la complejidad del modelo
}

In [8]:
# Configurar RandomizedSearchCV
randomized_search_gbt = RandomizedSearchCV(
    estimator=gbt_model,
    param_distributions=param_gbt,
    n_iter=10,  
    cv=5, 
    scoring='neg_mean_squared_error',  
    n_jobs=-1, 
    random_state=42  
)


In [9]:
# Ajustar el modelo
randomized_search_gbt.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
                   param_distributions={'ccp_alpha': [0.0, 0.1],
                                        'learning_rate': [0.01, 0.05, 0.1],
                                        'loss': ['squared_error', 'huber'],
                                        'max_depth': [3, 5, 7],
                                        'max_features': ['sqrt', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.8, 1.0]},
                   random_state=42, scoring='neg_mean_squared_error')

In [10]:
# Obtener los mejores parámetros y la mejor puntuación
best_params_gbt = randomized_search_gbt.best_params_
best_score_gbt = randomized_search_gbt.best_score_

print("Mejores parámetros:", best_params_gbt)
print("Mejor puntuación:", best_score_gbt)

Mejores parámetros: {'subsample': 1.0, 'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 7, 'loss': 'squared_error', 'learning_rate': 0.01, 'ccp_alpha': 0.1}
Mejor puntuación: -332.83817289389054


In [11]:
# Crear una nueva instancia de GradientBoostingRegressor con los mejores parámetros encontrados
best_gbt_model = GradientBoostingRegressor(**randomized_search_gbt.best_params_)

best_gbt_model.fit(X_train, y_train)

y_pred = best_gbt_model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calcular R^2
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calcular RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Mean Squared Error: 229.77524916163918
R^2 Score: 0.4648886757428349
Root Mean Squared Error: 15.158339261332001


In [12]:
model_scores = {
    'model': 'GradientBoostedRegresion',
    'mse': mse,
    'r2': r2,
    'rmse': rmse
}
print(model_scores)

{'model': 'GradientBoostedRegresion', 'mse': 229.77524916163918, 'r2': 0.4648886757428349, 'rmse': 15.158339261332001}


In [13]:
def save_scores(model_scores: dict):
  status = []
  #Utilizamos un try-except en caso de que el archivo no exista que cree uno
  try:
    #Si el archivo existe continúa por acá
    scores = pd.read_csv('scores.csv', sep=';')
    status.append('El archivo existe')
    exist = model_scores['model'] in scores['model'].values
    #Validamos si el modelo ya existe en el archivo de scores
    if (exist):
      #Si el modelo ya existe reemplazamos sus valores
      status.append('El modelo existe')
      criteria = scores['model'] == model_scores['model']
      index = scores[criteria].index[0]
      scores.iloc[index] = model_scores
      status.append('Se reemplazaron los valores del modelo')
    else:
      #Si el modelo no existe lo agregamos
      status.append('El modelo no existe')
      df_model_scores = pd.DataFrame(model_scores, index=[0])
      scores = pd.concat([scores, df_model_scores], ignore_index=True)
      status.append('Se añadió modelo nuevo y sus valores')
  except:
    #Si no existe el archivo lo creamos y cargamos los scores del modelo
    status.append('El archivo no existe')
    scores = pd.DataFrame(model_scores, index=[0])

  status.append('Se sobrescrive el archivo scores.csv con valores nuevos')
  scores.to_csv('scores.csv', sep=';', index=False)
  return status

In [14]:
save_scores(model_scores)